In [6]:
using System.Threading;
using System.Threading.Tasks;

In [7]:
#nullable enable

public class WaitHandleTestObj
{
    private AutoResetEvent autoResetEvent;
    private CancellationTokenSource? ctkSource;

    public WaitHandleTestObj()
    {
        autoResetEvent = new AutoResetEvent(false);
    }

    public void Run()
    {
        ctkSource?.Dispose();
        ctkSource = null;
        ctkSource
            = new CancellationTokenSource();

        CancellationToken ct
            = ctkSource.Token;

        autoResetEvent.Reset();

        Task.Run(() =>
        {
            var wh = new WaitHandle[] { autoResetEvent, ct.WaitHandle };

            while (true)
            {
                // do something

                Console.WriteLine("Before wait.");
                int signaledIndex = WaitHandle.WaitAny(wh);
                //int signaledIndex = WaitHandle.WaitAny(wh, 1000);

                // The cancellation token wait handle was signaled
                if (signaledIndex == 0)
                {
                    Console.WriteLine("autoResetEvent signaled.");
                }
                else if (signaledIndex == 1)
                {
                    Console.WriteLine("cancellation token signaled.");
                    break;
                }
                else if (signaledIndex == WaitHandle.WaitTimeout)
                {
                    Console.WriteLine("timeout signaled.");
                }

                Thread.Sleep(100);
            }

            Console.WriteLine("Task end.");
        });
    }

    public void Set()
    {
        autoResetEvent.Set();
    }

    public void Cancel()
    {
        ctkSource?.Cancel();
    }
}

In [8]:
var tmpObj = new WaitHandleTestObj();
tmpObj.Run();

await Task.Delay(2000);

tmpObj.Set();

Before wait.
autoResetEvent signaled.


In [9]:
var tmpObj2 = new WaitHandleTestObj();
tmpObj2.Run();

await Task.Delay(2000);

tmpObj2.Cancel();

Before wait.
cancellation token signaled.
Task end.
